In [20]:
import requests
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
import sys
from PIL import Image, ImageDraw, ImageFont
import json
import cv2
import os
import time
import tensorflow as tf
from keras import layers, Input, models, optimizers
from keras.models import Model, Sequential
from keras.models import load_model
from keras import layers, Input, models, optimizers
from keras.layers import MaxPooling2D, Conv2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
import random

# 학습된 모델 불러오기

In [22]:
model_pattern = './model/pattern_9985.h5'
var_pattern =  ['Check', 'Dot', 'Floral', 'Graphic', 'Leopard', 'None', 'Stripe']

model_top = './model/upper_9978.h5'
var_top = ['Hoodie', 'Long-sleeve-tee', 'Shirts', 'Short-sleeve-tee', 'Sweater',]

model_bottom = './model/lower0425_9990.h5'
var_bottom = ['Chiffon-skirt', 'Cotten-long-pants', 'Cutoffs', 'H-line-skirt', 'Jeans']

model_style = './model/style_model_1_90,63.h5'
var_style =  ['Casual', 'Feminine', 'Formal']

In [23]:
conv_base = VGG16(weights = 'imagenet',include_top=False,input_shape=(150,150,3))

top_model = models.Sequential()
top_model.add(layers.Dense(256, activation='relu', input_dim = 4 * 4 * 512))
top_model.add(layers.Dropout(0.5))
top_model.add(layers.Dense(4, activation='softmax'))
top_model.compile(optimizer = optimizers.RMSprop(lr=2e-5),
                      loss='categorical_crossentropy',
                      metrics=['acc'])
top_model.load_weights(model_top)

In [24]:
pattern_model = models.Sequential()
pattern_model.add(layers.Dense(256, activation='relu', input_dim = 4 * 4 * 512))
pattern_model.add(layers.Dropout(0.5))
pattern_model.add(layers.Dense(7, activation='softmax'))
pattern_model.compile(optimizer = optimizers.RMSprop(lr=2e-5),
                      loss='categorical_crossentropy',
                      metrics=['acc'])
pattern_model.load_weights(model_pattern)

In [25]:
model_bottom = load_model(model_bottom)
model_style = load_model(model_style)

# 사용자가 입력한 이미지(옷) 상하의 구분

In [26]:
def identify_clothes(data,up_lo): # 사진 데이터과 상의 이지 하의 인지 상의 :0
    if up_lo==0: # 상의 라면
        p = conv_base.predict(data)
        p = p.reshape((-1,8192))
        p = pattern_model.predict_classes(p)[0]
        tp = var_pattern[p]
        
        s = conv_base.predict(data)
        s = s.reshape((-1,8192))
        s = top_model.predict_classes(s)[0]
        ts = var_top[s]
        
        return tp, ts
    else: # 하의 라면
        p = conv_base.predict(data)
        p = p.reshape((-1,8192))
        p = pattern_model.predict_classes(p)[0]
        bp = var_pattern[p]           
        
        s = conv_base.predict(data)
        s = s.reshape((-1,8192))
        s = model_bottom.predict_classes(s)[0]
        bs = var_bottom[s]
        
        return bp, bs
        

# 옷의 pattern, shape 및 style에 맞는 옷 추천

In [27]:
def recommend(df, style, pattern, shape): # dataframe  스타일, identify_clothes(data,0)[0], identify_clothes(data,0)[1]
    if style == 'Casual':
        ca_df = df[df['style']=='Casual']
        ca_df = ca_df[['top shape','top pattern', 'bottom shape', 'bottom pattern']]
        ca_df = ca_df.dropna(axis = 0)
        ca_df['zeros'] = 1
        ca_table=ca_df.pivot_table('zeros',index=['top shape','top pattern'],columns=['bottom shape', 'bottom pattern'], aggfunc='count')
        table = ca_table.fillna(0)
        
    elif style =="Feminine":
        fe_df = df[df['style']=='Feminine']
        fe_df = fe_df[['top shape','top pattern', 'bottom shape', 'bottom pattern']]
        fe_df = fe_df.dropna(axis = 0)
        fe_df['zeros'] = 1
        fe_table= fe_df.pivot_table('zeros',index=['top shape','top pattern'],columns=['bottom shape', 'bottom pattern'], aggfunc='count')
        table = fe_table.fillna(0)
        
    else:
        fo_df = df[df['style']=='Formal']
        fo_df = fo_df[['top shape','top pattern', 'bottom shape', 'bottom pattern']]
        fo_df = fo_df.dropna(axis = 0)
        fo_df['zeros'] = 1
        fo_table= fo_df.pivot_table('zeros',index=['top shape','top pattern'],columns=['bottom shape', 'bottom pattern'], aggfunc='count')
        table = fo_table.fillna(0)
        
        
    if shape in ['Hoodie', 'Long-sleeve-tee', 'Shirts', 'Short-sleeve-tee', 'Sweater']:
        matched_lower = table.loc[(shape, pattern)].sort_values(ascending=False).index[0]
        matched_shape = matched_lower[0]
        matched_pattern = matched_lower[1]
    else :
        matched_upper = table.xs((shape,pattern), axis=1).sort_values(ascending=False).index[0]
        matched_shape = matched_upper[0]
        matched_pattern = matched_upper[1]
        
#     if matched_shape == 'Chiffon-skirt':
#         matched_shape = 'A-line-skirt'
        
    return matched_shape, matched_pattern            
            

# 추천 메트릭스

In [28]:
df = pd.read_csv('./data/matrix_except_crawling.csv')
df.head(5)
df.shape

(2762, 8)

In [29]:
for _ in range(len(df)):
    try:
        k = df['img_dir'].iloc[_]
        
        if k.split('.')[-1] != 'jpg':
            df = df.drop([_])   # 오류 나긴하는데 잘됨
    except:
        pass

In [30]:
ca_df = df[df['style']=='Casual']
ca_df = ca_df[['top shape','top pattern', 'bottom shape', 'bottom pattern']]
ca_df = ca_df.dropna(axis = 0)
ca_df['zeros'] = 1
ca_table=ca_df.pivot_table('zeros',index=['top shape','top pattern'],columns=['bottom shape', 'bottom pattern'], aggfunc='count')
table = ca_table.fillna(0)

In [31]:
table.to_csv('ca_table.csv', header=True, index=True)

In [32]:
fe_df = df[df['style']=='Feminine']
fe_df = fe_df[['top shape','top pattern', 'bottom shape', 'bottom pattern']]
fe_df = fe_df.dropna(axis = 0)
fe_df['zeros'] = 1
fe_table= fe_df.pivot_table('zeros',index=['top shape','top pattern'],columns=['bottom shape', 'bottom pattern'], aggfunc='count')
table = fe_table.fillna(0)

In [33]:
table.to_csv('fe_table.csv', header=True, index=True)

In [34]:
fo_df = df[df['style']=='Formal']
fo_df = fo_df[['top shape','top pattern', 'bottom shape', 'bottom pattern']]
fo_df = fo_df.dropna(axis = 0)
fo_df['zeros'] = 1
fo_table= fo_df.pivot_table('zeros',index=['top shape','top pattern'],columns=['bottom shape', 'bottom pattern'], aggfunc='count')
table = fo_table.fillna(0)

In [35]:
table.to_csv('fo_table.csv', header=True, index=True)

# 이미지 넣어 구현하기

In [36]:
def show_recommand_img(img, style, num):
        #plt.imshow(img)
        img__ = Image.open('start.jpg')
        img__.show()
        img.show()
        img = img.resize((150,150))
        data = np.array(img)
        data = data.astype('float')
        data = data/255
        data = data.reshape((1,150,150,3))
        
        print('옷의 패턴 : {} \n옷의 형태 : {}'.format(identify_clothes(data,1)[0],identify_clothes(data,1)[1]))
        
        print('스타일은: {} \n'.format(style))
        
        img_ = Image.open('show.jpg')
        img_.show()
        shape, pattern = recommend(df, style, identify_clothes(data,1)[0], identify_clothes(data,1)[1])
        
        recommend_folder_dir = './data/recommend/' + shape + '_' + pattern + '/'
        a = os.listdir(recommend_folder_dir)
        random.shuffle(a)
        num = int(num)
        for img_name in a[:num]:
            image = Image.open(recommend_folder_dir + img_name)
            #plt.imshow(image)
            image.show()
        

In [43]:
print('=== 이미지를 입력해주세요 ===')
img_dir = input() 
print('\n')
print('=== Feminine, Casual, Formal 스타일을 입력해주세요 ===')
style = input()
print('\n')
print('=== 추천 받을 옷의 갯수를 입력해주세요 ===')
num = input()
print('\n')
print('=== 옷 분석 및 추천 ===')
img = Image.open(img_dir)
show_recommand_img(img,style,num)

=== 이미지를 입력해주세요 ===
./test/04.jpg


=== Feminine, Casual, Formal 스타일을 입력해주세요 ===
Feminine


=== 추천 받을 옷의 갯수를 입력해주세요 ===
3


=== 옷 분석 및 추천 ===
옷의 패턴 : None 
옷의 형태 : Cotten-long-pants
스타일은: Feminine 

